In [1]:
import os
import pandas as pd
import sqlite3
import csv
import requests
import json

# 1. 3rd Party API Data Ingestion

In [2]:
r = requests.get("https://api.le-systeme-solaire.net/rest/bodies")

- name
- mass{massValue, massExponent} 
- vol{volValue, volExponent}
- gravity
- discoveryDate

## First try

In [3]:
bodies = r.json()["bodies"]

solarSystemPlanets = [{"name":body["name"],
                      "mass_Value":body["mass"]["massValue"],
                      "mass_Exponent":body["mass"]["massExponent"],
                      "vol_Value":body["vol"]["volValue"],
                      "vol_Exponent":body["vol"]["volExponent"], 
                      "gravity":body["gravity"],
                      "discoveryDate":body["discoveryDate"]}
                      for body in bodies if body["isPlanet"] == True]
planet_df = pd.DataFrame(solarSystemPlanets)
planet_df

Because of the nature of the problem, we can use, the following approach in which we concatenate name and gravity. 

I'm unsure If the below was the expected from the question. But I'll create Mass and Volume columns

In [4]:
planet_df["uniqKey"] = planet_df["name"].str.cat(planet_df["gravity"].astype(str),sep="-")

planet_df["Mass"] = planet_df["mass_Value"]*pow(10,planet_df["mass_Exponent"])
planet_df["Volume"] = planet_df["vol_Value"]*pow(10,planet_df["vol_Exponent"])
planet_df

,name,mass_Value,mass_Exponent,vol_Value,vol_Exponent,gravity,discoveryDate,uniqKey,Mass,Volume
0,(1) Cérès,9.39300,20,4.21000,9,0.280,01/01/1801,(1) Cérès-0.28,7.294866e+19,4.210000e+09
1,(136199) Éris,1.66000,22,6.59000,9,0.820,05/01/2005,(136199) Éris-0.82,3.095422e+18,6.590000e+09
2,Uranus,8.68127,25,6.83300,13,8.870,13/03/1781,Uranus-8.87,1.381101e+19,6.833000e+13
3,Pluton,1.30300,22,7.15000,9,0.620,18/02/1930,Pluton-0.62,2.429720e+18,7.150000e+09
4,Neptune,1.02413,26,6.25400,13,11.150,23/09/1846,Neptune-11.15,-2.599001e+18,6.254000e+13
5,(136108) Hauméa,4.00600,21,2.40000,9,0.401,28/12/2004,(136108) Hauméa-0.401,1.552653e+19,2.400000e+09
6,(136472) Makémaké,4.40000,21,1.70000,9,0.500,31/03/2005,(136472) Makémaké-0.5,1.705361e+19,1.700000e+09
7,Jupiter,1.89819,27,1.43128,15,24.790,,Jupiter-24.79,-1.315616e+19,1.431280e+15
8,Mars,6.41712,23,1.63180,11,3.710,,Mars-3.71,1.285840e+18,1.631800e+11
9,Mercure,3.30114,23,6.08300,10,3.700,,Mercure-3.7,6.614706e+17,6.083000e+10


# ------------------------------------------------------------------------------------------

#### This calculation leads to different results on my side

In [5]:
(planet_df["mass_Value"][0])*(10**planet_df["mass_Exponent"][0])
print(planet_df["mass_Value"][0])
print(planet_df["mass_Exponent"][0])
print(9.393*10**20)
print(planet_df["mass_Value"][0]*(10**planet_df["mass_Exponent"][0]))

9.393
20
9.393000000000001e+20
7.294866457823091e+19


# ------------------------------------------------------------------------------------------

## Final try

### 1. Query the Name, Mass, Volume, Gravity and Discovery Date.

In [5]:
# Workaround doing corresponding calculations outside of pandas
bodies2 = r.json()["bodies"]

solarSystemPlanets2 = [{"name":body["name"],
                      "Mass":body["mass"]["massValue"] * 10 ** body["mass"]["massExponent"],
                      "Volume":body["vol"]["volValue"] * 10 ** body["vol"]["volExponent"], 
                      "gravity":body["gravity"],
                      "discoveryDate":body["discoveryDate"]}
                      for body in bodies if body["isPlanet"] == True]
planet_df2 = pd.DataFrame(solarSystemPlanets2)
planet_df2

,name,Mass,Volume,gravity,discoveryDate
0,(1) Cérès,9.393000e+20,4.210000e+09,0.280,01/01/1801
1,(136199) Éris,1.660000e+22,6.590000e+09,0.820,05/01/2005
2,Uranus,8.681270e+25,6.833000e+13,8.870,13/03/1781
3,Pluton,1.303000e+22,7.150000e+09,0.620,18/02/1930
4,Neptune,1.024130e+26,6.254000e+13,11.150,23/09/1846
5,(136108) Hauméa,4.006000e+21,2.400000e+09,0.401,28/12/2004
6,(136472) Makémaké,4.400000e+21,1.700000e+09,0.500,31/03/2005
7,Jupiter,1.898190e+27,1.431280e+15,24.790,
8,Mars,6.417120e+23,1.631800e+11,3.710,
9,Mercure,3.301140e+23,6.083000e+10,3.700,


### 2. Assign an unique key

In [6]:
planet_df2["uniqKey"] = planet_df2["name"].str.cat(planet_df2["gravity"].astype(str),sep="-")

### 3. Volume ranking: Give each planet a volume rank from largest to smallest volume (1 =
### largest)

In [7]:
planet_df2["Volume_ranking"] = planet_df2["Volume"].rank(method='dense', ascending=False)
planet_df2

,name,Mass,Volume,gravity,discoveryDate,uniqKey,Volume_ranking
0,(1) Cérès,9.393000e+20,4.210000e+09,0.280,01/01/1801,(1) Cérès-0.28,11.0
1,(136199) Éris,1.660000e+22,6.590000e+09,0.820,05/01/2005,(136199) Éris-0.82,10.0
2,Uranus,8.681270e+25,6.833000e+13,8.870,13/03/1781,Uranus-8.87,3.0
3,Pluton,1.303000e+22,7.150000e+09,0.620,18/02/1930,Pluton-0.62,9.0
4,Neptune,1.024130e+26,6.254000e+13,11.150,23/09/1846,Neptune-11.15,4.0
5,(136108) Hauméa,4.006000e+21,2.400000e+09,0.401,28/12/2004,(136108) Hauméa-0.401,12.0
6,(136472) Makémaké,4.400000e+21,1.700000e+09,0.500,31/03/2005,(136472) Makémaké-0.5,13.0
7,Jupiter,1.898190e+27,1.431280e+15,24.790,,Jupiter-24.79,1.0
8,Mars,6.417120e+23,1.631800e+11,3.710,,Mars-3.71,7.0
9,Mercure,3.301140e+23,6.083000e+10,3.700,,Mercure-3.7,8.0


### 4. Output a json document ready with the data for all planets formatted as mentioned
### above

In [8]:
jsonOutput = planet_df2.to_json(force_ascii=False)
parsed = json.loads(jsonOutput)

f = open("jsonPlanetTable.txt", "w")
f.write(json.dumps(parsed, indent=4,ensure_ascii=False))
f.close()

In [9]:
parsed_records = json.loads(planet_df2.to_json(orient="records"))
f = open("jsonPlanetRecords.txt", "w")
f.write(json.dumps(parsed_records , indent=4,ensure_ascii=False))
f.close()

# 2. SQL

Below code can load data from a csv format with constraints in columns, create a database, get the answer from the table and output it as a dataframe. 

This can be useful for further processing. Queries are basically snapshots, but with these dataframe you can create further files that can be useful for other needs.

In [10]:
def sqlCreate(db_name):
    df = pd.read_csv("zeeto2.csv")
    try:
        con = sqlite3.connect(db_name)
        print("Connection is established: Database zeeto created")
        cur = createTable(con,df)
        return(con,cur)
    except sqlite3.Error as e:
        print(e)
def createTable(con,df):
    cursor = con.cursor()
    cursor.execute("DROP TABLE IF EXISTS raw_campaign_events")
    table = """CREATE TABLE raw_campaign_events(timestamp_utc DATETIME NOT NULL,
    campaign_name TEXT NOT NULL,
    event_type TEXT NOT NULL,
    revenue REAL DEFAULT NULL);"""
    cursor.execute(table)
    insertValue(cursor,df)
    con.commit()
    return(cursor)
def insertValue(cursor,df):
    sql = "INSERT INTO `raw_campaign_events` "+str(tuple(df))+" VALUES (?, ?, ?, ?)"
    records = df.values.tolist()
    cursor.executemany(sql,records)
#We create initial conditions
con,cur = sqlCreate("zeeto")

Connection is established: Database zeeto created


In [11]:
sqlAnswer = """SELECT r.campaign_name, r.visits, r.impressions, i.last_impressed_at_pst, r.conversions, c.last_converted_at_pst, tr.total_revenue
FROM 
(SELECT campaign_name, 
count(CASE event_type WHEN 'visit_created' THEN 1 END) as visits,
count(CASE event_type WHEN 'campaign_impressed' THEN 1 END) as impressions, 
count(CASE event_type WHEN 'campaign_converted' THEN 1 END) as conversions from raw_campaign_events group by campaign_name) r 
LEFT join 
(SELECT campaign_name, MAX(datetime(timestamp_utc||'+08:00')) as last_impressed_at_pst FROM raw_campaign_events where event_type = 'campaign_impressed' group by campaign_name,event_type) i
on r.campaign_name = i.campaign_name
LEFT join 
(SELECT campaign_name, MAX(datetime(timestamp_utc||'+08:00')) as last_converted_at_pst FROM raw_campaign_events where event_type = 'campaign_converted' group by campaign_name,event_type) c
on r.campaign_name = c.campaign_name
LEFT JOIN
(SELECT campaign_name, sum(revenue) as total_revenue FROM raw_campaign_events where event_type = 'campaign_converted' group by campaign_name) tr
on r.campaign_name = tr.campaign_name
group by r.campaign_name;"""
df = pd.read_sql_query(sqlAnswer, con)
df

,campaign_name,visits,impressions,last_impressed_at_pst,conversions,last_converted_at_pst,total_revenue
0,Campaign a,1,2,2021-01-01 04:04:19,1,2021-01-01 04:03:16,0.99
1,Campaign b,1,1,2021-01-01 04:01:55,0,None,NaN
2,Campaign c,1,1,2021-01-01 04:07:15,1,2021-01-01 04:10:52,1.20
